![](https://storage.googleapis.com/mle-courses-prod/users/61b6fa1ba83a7e37c8309756/private-files/a611c990-b394-11ef-99fc-e9ac20f73f15-RAGGraph___Neo4J___CamelAI__5_.png)

Tác giả: [Protonx Team](https://protonx.coursemind.io/courses/66b04dce02b797001269359b/info)

In [ ]:
!pip install --q datasets
!pip install --q --upgrade trl
!pip install --upgrade accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 6.3 MB/s eta 0:00:00


In [ ]:
!pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 33.1 MB/s eta 0:00:00


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

In [ ]:
max_seq_len = 100

In [ ]:
model_id = "imthanhlv/vigpt2medium"

In [ ]:
# model.config.pretraining_tp = 1

tokenizer = AutoTokenizer.from_pretrained(model_id)
if tokenizer.pad_token_id is None:
    tokenizer.pad_token = tokenizer.eos_token

tokenizer.padding_side = "right"

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/207 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/781k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/439k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

In [ ]:
import random

from datasets import load_dataset
open_instruct_dataset = load_dataset("5CD-AI/Vietnamese-Ecommerce-Alpaca", split="train")


total_data_points = len(open_instruct_dataset)
sample_size = 10000
random_indices = random.sample(range(total_data_points), sample_size)



# filter dataset to rows where the entire context length is less than or equal to max_seq_len,
# which is the size of the DeciLM-6B context window

open_instruct_dataset = open_instruct_dataset.select(random_indices)

dataset = open_instruct_dataset.filter(
    lambda example: (len(tokenizer(example.get("instruction", ""), truncation=True)["input_ids"]) +
                     len(tokenizer(example.get("input", ""), truncation=True)["input_ids"]) +
                     len(tokenizer(example.get("output", ""), truncation=True)["input_ids"])) <= max_seq_len)


ecommerce_alpaca.json:   0%|          | 0.00/84.9M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/69303 [00:00<?, ? examples/s]

Filter:   0%|          | 0/10000 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [ ]:
dataset.to_pandas()[:10]


,output,instruction,input
0,Khăn giấy ướt cho bé Bobby thường được phân lo...,Phân loại sản phẩm khăn giấy ướt cho bé sau đâ...,
1,Một ví dụ về câu thành ngữ liên quan đến dụng ...,Đưa ra ví dụ về câu thành ngữ liên quan đến dụ...,
2,1. Độ sáng cao: Độ sáng cao đảm bảo hình ảnh c...,Tạo danh sách ba giá trị mà Máy chiếu nên có.,
3,"Phù rể có thể giúp chú rể mặc quần áo, chỉnh s...",Tạo ra câu mô tả một hành động có thể thực hiệ...,
4,Chúng tôi có chính sách đổi trả trong vòng 30 ...,Bạn đang bán một chiếc áo khoác nữ màu đỏ với ...,
5,Hướng dẫn này là đạo đức miễn là việc sản xuất...,"Với những hướng dẫn sau, hãy quyết định xem cá...",
6,Máy làm tóc này tiện lợi như một người bạn đồn...,"Cho hai cụm từ, viết lại bằng cách sử dụng phé...",
7,3. Giải quyết vấn đề: là khả năng giải quyết v...,Xác định ba yếu tố quan trọng nhất của dịch vụ...,
8,"Dựa trên thông tin được cung cấp, dữ liệu thuộ...",Tìm hạng của dữ liệu đưa ra. Dữ liệu: kem đánh...,
9,Hút bụi cầm tay Philips FC6168/01 có những ưu ...,Tạo ra câu hỏi thông minh liên quan đến nội du...,


In [ ]:
dataset[0]

{'output': 'Khăn giấy ướt cho bé Bobby thường được phân loại là sản phẩm cao cấp, trong khi Khăn giấy ướt cho bé Huggies và Khăn giấy ướt cho bé Pampers được phân loại là sản phẩm trung bình.',
 'instruction': 'Phân loại sản phẩm khăn giấy ướt cho bé sau đây theo loại: cao cấp, trung bình, hoặc bình dân. Khăn giấy ướt cho bé Bobby, Khăn giấy ướt cho bé Huggies, Khăn giấy ướt cho bé Pampers.',
 'input': ''}

In [ ]:
def format_row_as_instruction_prompt(example):
    has_input = example.get('input', None) is not None

    if has_input:
        primer_prompt = ("Below is an instruction that describes a task, paired with an input "
                         "that provides further context. Write a response that appropriately completes the request.")
        input_template = f"### Input: \n{example['input']}\n\n"
    else:
        primer_prompt = ("Below is an instruction that describes a task. "
                         "Write a response that appropriately completes the request.")
        input_template = ""

    instruction_template = f"### Instruction: \n{example['instruction']}\n\n"

    if example.get('output', None):
        response_template = f"### Response: \n{example['output']}\n\n"
    else:
        response_template = ""

    # Wrap the resulting string in a list
    return [f"{primer_prompt}\n\n{instruction_template}{input_template}{response_template}"]




In [ ]:
print(format_row_as_instruction_prompt(dataset[0])[0])

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction: 
Phân loại sản phẩm khăn giấy ướt cho bé sau đây theo loại: cao cấp, trung bình, hoặc bình dân. Khăn giấy ướt cho bé Bobby, Khăn giấy ướt cho bé Huggies, Khăn giấy ướt cho bé Pampers.

### Input: 


### Response: 
Khăn giấy ướt cho bé Bobby thường được phân loại là sản phẩm cao cấp, trong khi Khăn giấy ướt cho bé Huggies và Khăn giấy ướt cho bé Pampers được phân loại là sản phẩm trung bình.




In [ ]:
print(format_row_as_instruction_prompt(dataset[1])[0])

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction: 
Đưa ra ví dụ về câu thành ngữ liên quan đến dụng cụ làm đẹp.

### Input: 


### Response: 
Một ví dụ về câu thành ngữ liên quan đến dụng cụ làm đẹp là: "Cái răng cái tóc là góc con người". Câu thành ngữ này cho thấy rằng việc chăm sóc tóc và răng miệng là rất quan trọng đối với vẻ đẹp của một người, và người ta nên dành thời gian và công sức để chăm sóc chúng.




In [ ]:
from trl import SFTConfig, SFTTrainer, DataCollatorForCompletionOnlyLM

response_template = "### Response:"
collator = DataCollatorForCompletionOnlyLM(response_template, tokenizer=tokenizer)


In [ ]:
import torch

In [ ]:
from sklearn.model_selection import train_test_split

# Split dataset into training and validation sets
train_indices, val_indices = train_test_split(range(len(dataset)), test_size=0.1, random_state=42)
train_dataset = dataset.select(train_indices)
val_dataset = dataset.select(val_indices)


from datasets import Dataset
from sklearn.model_selection import train_test_split

# Step 1: Format the Dataset
def format_dataset(dataset):
    """
    Format each row in the dataset using the format_row_as_instruction_prompt function
    and return a Hugging Face Dataset object.
    """
    # Create a dictionary to store formatted data
    formatted_data = {"text": []}

    for example in dataset:
        formatted = format_row_as_instruction_prompt(example)[0]
        formatted_data["text"].append(formatted)

    # Convert the dictionary into a Hugging Face Dataset
    return Dataset.from_dict(formatted_data)


# Step 2: Split the Dataset
def split_dataset(dataset, test_size=0.1, random_seed=42):
    """
    Split the dataset into training and validation sets.
    """
    train_indices, val_indices = train_test_split(range(len(dataset)), test_size=test_size, random_state=random_seed)
    train_dataset = dataset.select(train_indices)
    val_dataset = dataset.select(val_indices)
    return train_dataset, val_dataset

# Format the dataset
formatted_dataset = format_dataset(dataset)

# Split into train and validation datasets
train_dataset, val_dataset = split_dataset(formatted_dataset)

# Print a sample for verification
print(f"Sample formatted training example:\n{train_dataset[0]}")
print(f"Sample formatted validation example:\n{val_dataset[0]}")


Sample formatted training example:
{'text': 'Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction: \nHãy cho tôi một ví dụ về một hashtag có thể được sử dụng cho một công ty cung cấp sản phẩm về ngoại thất sân vườn.\n\n### Input: \n\n\n### Response: \nMột ví dụ về hashtag có thể được sử dụng cho một công ty cung cấp sản phẩm về ngoại thất sân vườn là #NgoạiThấtĐẹp.\n\n'}
Sample formatted validation example:
{'text': 'Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction: \nHãy xác định âm đôi trong câu này: "Túi đeo chéo nam đẹp tuyệt vời".\n\n### Input: \n\n\n### Response: \nÂm đôi trong câu này là "tuyệt" và "vời". Âm đôi là những từ có âm thanh giống nhau nhưng có cách viết và ý nghĩa khác nhau.\n\n'}


In [ ]:
!pip install evaluate

import evaluate

# Load a metric
metric = evaluate.load("accuracy")  # Replace "accuracy" with the metric you need

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    # Convert logits to predicted token indices
    predictions = logits.argmax(axis=-1)

    # Mask out ignored labels (e.g., -100)
    valid_indices = labels != -100

    # Flatten predictions and labels while filtering valid indices
    predictions = predictions[valid_indices]
    labels = labels[valid_indices]

    print("Predictions:", predictions)
    print("Labels:", labels)

    # Compute the metric
    return metric.compute(predictions=predictions, references=labels)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 590.9 kB/s eta 0:00:00


In [ ]:
max_seq_len

100

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    use_cache=False,
    device_map="auto",
    trust_remote_code=True
)

model.resize_token_embeddings(len(tokenizer))

trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    args=SFTConfig(
        output_dir="/tmp",
        num_train_epochs=100,
        per_device_train_batch_size=1,
        per_device_eval_batch_size=1,
        gradient_accumulation_steps=10,
        evaluation_strategy="steps",  # Enable evaluation during training
        eval_steps=3,               # Evaluate every 100 steps
    ),
    formatting_func=None,  # Formatting is already done
    data_collator=collator,
)

trainer.train()


config.json:   0%|          | 0.00/958 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.44G [00:00<?, ?B/s]

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:309: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/2081 [00:00<?, ? examples/s]

Map:   0%|          | 0/232 [00:00<?, ? examples/s]

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: